## Logistic Regression implementation using TensorFlow
Use the TF framework to implement a Gradient Descent Logistic Regression algorithm!! We try utilizing the standard features TensorFlow offers such as saving the model and logging progress for tensorboard.

In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
from sklearn.datasets import make_moons

# Get data

X, y = make_moons()

rows,cols = X.shape
X_1 = np.c_[np.ones((rows,1)), X]

In [3]:

# Logisitc Regression class imlemented fully in TensorFlow obviously missing some standard functionality...
# but fitting logic has been fully implemented

from datetime import datetime

class LogisticReg_TF():
    
    # Creates computational graph
    def __init__(self, learning_rate=0.01):
        tf.reset_default_graph()
        
        self.data = tf.placeholder(tf.float32, shape=(None,(cols+1)), name='data')
        self.classes = tf.placeholder(tf.float32, shape=(None,1), name='classes')
        self.theta = tf.Variable(tf.random_uniform([cols + 1,1],-1,1), name='theta')
        
        # y = 1/(1 + e^-(w * X))
        self.pred = tf.divide(tf.constant(1.0,dtype=tf.float32),
                        tf.math.add(
                            tf.constant(1.0,dtype=tf.float32),
                            tf.math.exp(tf.multiply(
                                tf.constant(-1.0,dtype=tf.float32),
                                tf.matmul(self.data,self.theta)))
                        ), name='pred')
        
        # loss = -(y * log(pred) + (1-y) * log(pred))
        self.loss = tf.multiply(
                        tf.constant(-1.0,dtype=tf.float32),
                        tf.math.add(
                            tf.matmul(tf.transpose(self.classes),
                                      tf.math.log(self.pred)),
                            tf.matmul(tf.transpose(tf.subtract(tf.constant(1.0,dtype=tf.float32),self.classes)),
                                      tf.math.log(self.pred))
                        ), name='loss')
        
        # gradient descent update to theta 
        self.train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(self.loss, var_list=[self.theta])
    
    
    # Entire fit process
    def fit(self, X, y, n_epochs):
        
        # Logging tools
        now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        log_dir = os.path.join(os.getcwd(), 'tensorflow/logs/run-{}/'.format(now))
        
        loss_summary = tf.summary.scalar('Log Loss',tf.squeeze(self.loss))
        writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())
        
        # Creating session
        init = tf.global_variables_initializer()
        ss = tf.InteractiveSession()
        ss.run(init)
        saver = tf.train.Saver()
        
        # GD update
        for epoch in range(n_epochs):
            ss.run(log.train, feed_dict={log.data:np.reshape(X,(-1,3)),log.classes:np.reshape(y,(-1,1))})
    
            # Log updates
            if epoch % 5 == 0:
                saver.save(ss, os.path.join(os.getcwd(), 'tensorflow/models/log_reg.ckpt'))
                log_str = loss_summary.eval(feed_dict={self.data:np.reshape(X_1,(-1,3)),self.classes:np.reshape(y,(-1,1))})
                writer.add_summary(log_str, epoch)
    
        ss.close()

In [4]:
# Run it!

log = LogisticReg_TF(0.01)
log.fit(X_1, y, 51)

INFO:tensorflow:Summary name Log Loss is illegal; using Log_Loss instead.
